# Predicting a Tabular Transformation from Text
We plan on solving data transformation with a translation type approach.  There are two facets here:
1. Providing a better interface for data transformation
2. _Predicting_ which transformations are needed to get to a given schema

In the below notebook, I'm going to show a neural network that takes user-generated text as an input and predicts which transformation to use.  

**To be clear**, this is a proof of concept, and the UI is a work in progress.  We plan on training against a massive corpus of text and identifying features within the input to determine which columns to operate on.  Given the following user input:

```
Concatenate the first_name and last_name columns into a new column called full_name
```

Our new model would produce a sequence of transformations with the named columns as inputs, automatically.

While we're working out the kinks, a more basic demonstration will have to do:

In [98]:
import numpy as np
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

add_column_text = ["add a column","insert a column","place new column","add row numbers","insert row numbers"]
remove_column_text = ["remove column","delete column", "remove row numbers"]

augmented_add_column = []
for txt in add_column_text:
  for num in range(9):
    augmented_add_column.append(f'{txt} after column {num + 1}')
    augmented_add_column.append(f'{txt} before column {num + 1}')

augmented_remove_column = []
for txt in remove_column_text:
  for num in range(9):
    augmented_remove_column.append(f'{txt} after column {num + 1}')
    augmented_remove_column.append(f'{txt} before column {num + 1}')

dict = {}
dict['<unknown>'] = 0
dict['a'] = 1
dict['add'] = 2
dict['column'] = 3
dict['delete'] = 4
dict['insert'] = 5
dict['new'] = 6
dict['numbers'] = 7
dict['place'] = 8
dict['remove'] = 9
dict['row'] = 10

### Training / Test Data
We're processing user inputs into a six-word sequence.  We've augmented the data with text referencing where to perform the operation for the purposes of having a larger corpus of text.  These arguments won't be consumed in our demo for the sake of brevity.  The only feature engineering I'm going to perform is to encode our text as a one-hot vector with the limited vocabulary we've defined in the dictionary above.

In [99]:
def text_to_one_hot(text, indices):
  text_arr = text.split()
  text_arr += [''] * (seq_length - len(text_arr))
  arr = np.zeros((seq_length, len(indices)))
  for idx, token in enumerate(text_arr):
    if token in indices:
      oh_idx = indices[token]
    else:
      oh_idx = 0
    arr[idx][oh_idx] = 1.0
  return arr

Next, we're going to construct X and y vectors for use in training and testing our model:

In [100]:
# Transformations selected:
# [add_column, remove_column, unknown]
add_column_classification = np.array([1, 0, 0])
remove_column_classification = np.array([0, 1, 0])

add_column_y = np.full((len(augmented_add_column), 3), add_column_classification)
remove_column_y = np.full((len(augmented_remove_column), 3), remove_column_classification)
y = np.concatenate((add_column_y, remove_column_y), axis=0)

inputs = []
for text in augmented_add_column:
  input = text_to_one_hot(text, dict)
  inputs.append(input)

for text in augmented_remove_column:
  input = text_to_one_hot(text, dict)
  inputs.append(input)

X = np.array(inputs)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### LSTM Model
In order to classify this sequence of text, we're going to use a really basic LSTM model with a softmax output layer. The model will identify if the user wants to add a column or remove a column to this tabular dataset.

In [101]:
seq_length = 6
num_classifications = 3

model = Sequential()
model.add(LSTM(4, input_shape=(seq_length, len(dict))))
model.add(Dense(num_classifications, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=1, epochs=32)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_33 (LSTM)              (None, 4)                 256       
                                                                 
 dense_63 (Dense)            (None, 3)                 15        
                                                                 
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/32
115/115 [==============================] - 1s 2ms/step - loss: 1.0623 - accuracy: 0.4783
Epoch 2/32
115/115 [==============================] - 0s 2ms/step - loss: 0.9099 - accuracy: 0.6174
Epoch 3/32
115/115 [==============================] - 0s 1ms/step - loss: 0.7792 - accuracy: 0.6174
Epoch 4/32
115/115 [==============================] - 0s 2ms/step - loss: 0.6706 - accuracy: 0.6261
Epoch 5/32
115/115 [==========================

In [93]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 100.00%


### Next Steps

As you can see from this contrived example, we are 100% accurate and our work here is done.

Ok, it's not going to be that easy, but this is the start of a really intuitive interface for users building tabular data transformations.  

Aside from drastically improving the sophistication of this model, the next frontier is going to be interpreting data schemas and producing a **sequence of transformations** to translate existing data into the new schema.  Creating this automatic bridge would give us the ability for systems to exchange data with minimal user input.  